#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [1]:
import pandas as pd
import numpy as np

In [2]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-06 05:22:55--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  16.9MB/s    in 0.3s    

2025-02-06 05:22:55 (16.9 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [3]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [4]:
# прочитаем описание README
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией
         список идентификаторов пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификатор пользователя | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
        быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

'\nПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ\n==============================================\n\nВот краткое описание данных.\n\nml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:\n                gunzip ml-data.tar.gz\n                tar xvf ml-data.tar\n                mku.sh\n\nu.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.\n         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы\n         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.\n         Это разделенный табуляцией\n         список идентификаторов пользователей | элементов | оценок | временных меток.\n         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix\n\nu.info - Количество пользователей, элементов и оценок в наборе данных u.\n\nu.item - Информация о фильмах, которые будут показаны в этом разделе;\n         список фильмов, которые будут показаны в виде вкладок:\n      

In [5]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data', delimiter='\t')
u_data.columns = ['userID', 'movieID', 'rating', 'time']
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')

# так читать не хочет
# u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
# попробуем по-другому
import csv
csv_file = open('/content/unpacked_files/ml-100k/u.item', newline='', encoding='latin-1')
u_item = pd.read_csv(csv_file, delimiter='|')
u_item.columns = ['movieID', 'movie_title', 'release_date', 'video_release',
                  'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                  'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                  'Thriller', 'War', 'Western']
u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre', delimiter='|')
u_genre.columns = ['genre', '#']
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user', delimiter='|')
u_user.columns = ['userID', 'age', 'gender', 'occupation', 'p_code']
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')
u_occupation.columns = ['occupation']

In [6]:
# пользователь | элемент | рейтинг | временная метка
u_data.head()

,userID,movieID,rating,time
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [7]:
u_info.head()

,943 users
0,1682 items
1,100000 ratings


In [8]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
u_item_2 = u_item.copy()
u_item_2.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def genre_to_string (str, num):
  if num == 1:
    string = str

  return

In [11]:
 # добавим единую колонку жанров
 cols = ['unknown', 'Action', 'Adventure', 'Animation', 'Children_s', 'Comedy',
         'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
         'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

for index, row in u_item_2.iterrows():
    string = str()
    for ele in cols:
        if row[ele] == 1:
            if string == '':
                string += ele
            else:
                string += ('|'+ele)
        else:
            continue
    u_item.loc[index, 'genre'] = string

In [12]:
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action|Adventure|Thriller
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action|Comedy|Drama
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime|Drama|Thriller
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama


In [13]:
# посмотрим на список жанров
u_genre

,genre,#
0,Action,1
1,Adventure,2
2,Animation,3
3,Children's,4
4,Comedy,5
5,Crime,6
6,Documentary,7
7,Drama,8
8,Fantasy,9
9,Film-Noir,10


In [14]:
# пользователь | возраст | пол | род занятий | почтовый индекс
u_user.head()

,userID,age,gender,occupation,p_code
0,2,53,F,other,94043
1,3,23,M,writer,32067
2,4,24,M,technician,43537
3,5,33,F,other,15213
4,6,42,M,executive,98101


In [15]:
# профессии
u_occupation.head()

,occupation
0,artist
1,doctor
2,educator
3,engineer
4,entertainment


##LightFM (default)

In [94]:
import numpy as np

In [95]:
!pip install lightfm

In [96]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [97]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [98]:
data = fetch_movielens(min_rating=5.0)
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [99]:
train = movielens['train']
test = movielens['test']

Посмотрим на все это

In [100]:
train.toarray(), len(train.toarray())

(array([[5, 3, 4, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [5, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 5, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [101]:
test.toarray(), len(test.toarray())

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 943)

In [102]:
data = fetch_movielens()
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>


In [119]:
# зададим рейтинг
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [120]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [104]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.43, test 0.13.


In [105]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.62, test 0.23.


In [118]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [115]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]
        print(data['item_labels'])
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [110]:
print(known_positives)

NameError: name 'known_positives' is not defined

In [116]:
sample_recommendation(model, movielens, [10, 25, 451])

['T' 'G' 'F' ... 'S' 'Y' 'S']
User 10
     Known positives:
        B
        D
        S
     Recommended:
        R
        S
        E
['T' 'G' 'F' ... 'S' 'Y' 'S']
User 25
     Known positives:
        T
        T
        D
     Recommended:
        F
        L
        T
['T' 'G' 'F' ... 'S' 'Y' 'S']
User 451
     Known positives:
        T
        B
        P
     Recommended:
        R
        S
        E


##Соберем свой датафрейм

In [30]:
# объединим u_data и u_user
df_1 = u_data.merge(u_user, on='userID').reset_index(drop=True)
df_1.head()

,userID,movieID,rating,time,age,gender,occupation,p_code
0,186,302,3,891717742,39,F,executive,00000
1,22,377,1,878887116,25,M,writer,40206
2,244,51,2,880606923,28,M,technician,80525
3,166,346,1,886397596,47,M,educator,55113
4,298,474,4,884182806,44,M,executive,01581


In [31]:
# объединим df_1 и u_item
df_2 = df_1.merge(u_item, on='movieID').reset_index(drop=True)
df_2.head()

,userID,movieID,rating,time,age,gender,occupation,p_code,movie_title,release_date,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,3,891717742,39,F,executive,00000,L.A. Confidential (1997),01-Jan-1997,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,1,878887116,25,M,writer,40206,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,2,880606923,28,M,technician,80525,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,1,886397596,47,M,educator,55113,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,4,884182806,44,M,executive,01581,Dr. Strangelove or: How I Learned to Stop Worr...,01-Jan-1963,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [32]:
# выберем самое интересное
cols = ['userID', 'movieID', 'movie_title', 'rating', 'age', 'gender', 'occupation',
        'unknown', 'Action', 'Adventure', 'Animation',
        'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
        'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
        'Thriller', 'War', 'Western', 'genre']
df_data = df_2[cols]
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [33]:
# проверим жанр unknown
#df_data['unknown'].sum()

In [34]:
# посмотрим на эти строки
#df_data.loc[df_data['unknown'] == 1]

In [35]:
# считаю целесообразным удалить эти строки
#df_data.drop(df_data.index[df_data['unknown'] == 1], axis=0, inplace=True)
# уже так не считаю)

In [36]:
# проверим
#df_data.loc[df_data['unknown'] == 1]

In [37]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())
# посмотрим
list_occupations_unique,  len(list_occupations_unique)

(['executive',
  'writer',
  'technician',
  'educator',
  'engineer',
  'librarian',
  'programmer',
  'administrator',
  'student',
  'retired',
  'other',
  'doctor',
  'marketing',
  'artist',
  'lawyer',
  'salesman',
  'homemaker',
  'healthcare',
  'none',
  'entertainment',
  'scientist'],
 21)

In [38]:
df_data.head()

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War


In [39]:
#соберем объект жанров по фильмам
series_genre_movies = df_data['genre'].str.split('|')
# посмотрим
series_genre_movies[0], series_genre_movies, len(series_genre_movies)

(['Crime', 'Film-Noir', 'Mystery', 'Thriller'],
 0        [Crime, Film-Noir, Mystery, Thriller]
 1                         [Children_s, Comedy]
 2               [Drama, Romance, War, Western]
 3                               [Crime, Drama]
 4                                [Sci-Fi, War]
                          ...                  
 99271                                 [Comedy]
 99272                         [Comedy, Sci-Fi]
 99273                               [Thriller]
 99274                     [Children_s, Comedy]
 99275                                [Western]
 Name: genre, Length: 99276, dtype: object,
 99276)

##LightFM item_features например, genre_unique / user_features например, occupations_unique

In [40]:
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from lightfm.datasets import fetch_movielens

In [41]:
#df_data.reset_index()

In [42]:
df_data.head(2147)

,userID,movieID,movie_title,rating,age,gender,occupation,unknown,Action,Adventure,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,186,302,L.A. Confidential (1997),3,39,F,executive,0,0,0,...,1,0,0,1,0,0,1,0,0,Crime|Film-Noir|Mystery|Thriller
1,22,377,Heavyweights (1994),1,25,M,writer,0,0,0,...,0,0,0,0,0,0,0,0,0,Children_s|Comedy
2,244,51,Legends of the Fall (1994),2,28,M,technician,0,0,0,...,0,0,0,0,1,0,0,1,1,Drama|Romance|War|Western
3,166,346,Jackie Brown (1997),1,47,M,educator,0,0,0,...,0,0,0,0,0,0,0,0,0,Crime|Drama
4,298,474,Dr. Strangelove or: How I Learned to Stop Worr...,4,44,M,executive,0,0,0,...,0,0,0,0,0,1,0,1,0,Sci-Fi|War
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,239,10,Richard III (1995),5,39,M,artist,0,0,0,...,0,0,0,0,0,0,0,1,0,Drama|War
2143,238,111,"Truth About Cats & Dogs, The (1996)",4,42,F,administrator,0,0,0,...,0,0,0,0,1,0,0,0,0,Comedy|Romance
2144,130,267,unknown,5,20,M,none,1,0,0,...,0,0,0,0,0,0,0,0,0,unknown
2145,90,662,Somewhere in Time (1980),5,60,M,educator,0,0,0,...,0,0,0,0,1,0,0,0,0,Drama|Romance


In [43]:
#series_genre.isna().sum()

In [44]:
# соберем список уникальных жанров
series_genre = df_data['genre'].str.split('|')
list_genre_unique = list(set(np.concatenate(series_genre).ravel()))

list_genre_unique[:10], len(list_genre_unique)

(['War',
  'Children_s',
  'Documentary',
  'Sci-Fi',
  'Drama',
  'Horror',
  'Western',
  'Action',
  'Comedy',
  'unknown'],
 19)

In [45]:
# соберем список уникальных профессий
list_occupations_unique = list(df_data['occupation'].drop_duplicates())

list_occupations_unique[:5], len(list_occupations_unique)

(['executive', 'writer', 'technician', 'educator', 'engineer'], 21)

In [46]:
# соберем датасет
dataset = Dataset()
dataset.fit(users=df_data['userID'],
            items=df_data['movieID'],
            item_features=list_genre_unique,
            user_features=list_occupations_unique)

In [47]:
# соберем список уникальных пользователей и их профессии
df_unique_userID = df_data.drop_duplicates('userID')
list_user_features = [(x,[y]) for x,y in zip(df_unique_userID['userID'],
                      df_unique_userID['occupation'])
]

list_user_features[:10], len(list_user_features)

([(186, ['executive']),
  (22, ['writer']),
  (244, ['technician']),
  (166, ['educator']),
  (298, ['executive']),
  (115, ['engineer']),
  (253, ['librarian']),
  (305, ['programmer']),
  (6, ['executive']),
  (62, ['administrator'])],
 942)

In [48]:
# соберем список уникальных фильмов и их жанры
df_unique_movieID = df_data.drop_duplicates('movieID')
list_item_features = [(x,y) for x,y in zip(df_unique_movieID['movieID'], series_genre_movies)]

list_item_features[:10], len(list_item_features)

([(302, ['Crime', 'Film-Noir', 'Mystery', 'Thriller']),
  (377, ['Children_s', 'Comedy']),
  (51, ['Drama', 'Romance', 'War', 'Western']),
  (346, ['Crime', 'Drama']),
  (474, ['Sci-Fi', 'War']),
  (265, ['Action', 'Thriller']),
  (465, ['Adventure', 'Children_s', 'Romance']),
  (451, ['Comedy', 'Musical', 'Romance']),
  (86, ['Drama']),
  (257, ['Action', 'Adventure', 'Comedy', 'Sci-Fi'])],
 1681)

In [49]:
# подготовим sparse matrix
sm_user_features = dataset.build_user_features(list_user_features)
sm_user_features

<942x963 sparse matrix of type '<class 'numpy.float32'>'
	with 1884 stored elements in Compressed Sparse Row format>

In [50]:
# подготовим sparse matrix
sm_item_features = dataset.build_item_features(list_item_features)
sm_item_features

<1681x1700 sparse matrix of type '<class 'numpy.float32'>'
	with 5227 stored elements in Compressed Sparse Row format>

In [51]:
# подготовим данные о взаимодействии
sm_interactions, sm_weights = dataset.build_interactions(df_data[['userID',
                                                                  'movieID',
                                                                  'rating']].values)
sm_interactions

<942x1681 sparse matrix of type '<class 'numpy.int32'>'
	with 99276 stored elements in COOrdinate format>

Вместо использования исходных идентификаторов пользователей и элементов в нашем наборе данных LightFM внутренне присваивает каждому пользователю и элементу новый последовательный неотрицательный целочисленный идентификатор. Мы можем увидеть это сопоставление следующим образом:

In [52]:
user_id_map, user_feature_map, item_id_map, feature_item_map = dataset.mapping()

Посмотрим

In [53]:
list(user_id_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [54]:
list(user_feature_map.items())[:3]

[(186, 0), (22, 1), (244, 2)]

In [55]:
list(item_id_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [56]:
list(feature_item_map.items())[:3]

[(302, 0), (377, 1), (51, 2)]

In [57]:
# оценка эффективности
sm_train_interactions, sm_test_interactions = random_train_test_split(sm_interactions,
                                                                      test_percentage=0.2,
                                                                      random_state=42)
print(f'Shape of train interactions: {sm_train_interactions.shape}')
print(f'Shape of test interactions: {sm_test_interactions.shape}')

Shape of train interactions: (942, 1681)
Shape of test interactions: (942, 1681)


In [58]:
train.shape

(943, 1682)

In [59]:
test.shape

(943, 1682)

In [60]:
# построим модель
LEARNING_RATE = 0.05
EPOCHS = 100
COMPONENTS = 10
ITEM_ALPHA = 1e-6   # коэффициент упорядочения характеристик признаков
USER_ALPHA = 1e-6   # коэффициент упорядочения пользовательских функций

model = LightFM(loss='warp',
                no_components=COMPONENTS,
                learning_rate=LEARNING_RATE,
                item_alpha=ITEM_ALPHA,
                user_alpha=USER_ALPHA,
                random_state=42)

model.fit(interactions=sm_train_interactions,
          user_features=sm_user_features,
          item_features=sm_item_features,
          epochs=EPOCHS)

In [61]:
# оценим точность
np_arr_prec = precision_at_k(model,
                             test_interactions=sm_test_interactions,
                             user_features=sm_user_features,
                             item_features=sm_item_features)

print(len(np_arr_prec.shape))
np_arr_prec[:10]

1


array([0.2, 0.1, 0.1, 0.2, 0.1, 0.2, 0.2, 0.4, 0.3, 0.2], dtype=float32)

In [62]:
# найдем среднее значений
np_arr_prec.mean()

0.1245223

In [63]:
# рекомендация пользователя к товару
user_id = 63
list_scores = model.predict(user_id_map[user_id], list(item_id_map.values()))
print(len(list_scores))
list_scores[:5]

1681


array([  4.4328446 , -11.66942   ,  -9.5680685 ,  -0.44561297,
       -12.20017   ], dtype=float32)

In [64]:
# item_id_map — это словарь, который сопоставляет исходные идентификаторы фильмов
# с неотрицательными последовательными целыми числами, используемыми LightFM
# внутри системы
list(item_id_map)[:5]

[302, 377, 51, 346, 474]

In [65]:
# преобразуем список оценок рекомендаций в серию
series_scores = pd.Series(list_scores)
series_scores.index = item_id_map.keys()
print(len(list_scores))
series_scores[:5]

1681


,0
302,4.432845
377,-11.669420
51,-9.568069
346,-0.445613
474,-12.200170


In [66]:
# сортируем оценки в порядке убывания
series_scores.sort_values(ascending=False, inplace=True)
series_scores[:5]

,0
245,7.220359
328,4.510055
302,4.432845
332,4.330058
321,4.066901


In [71]:
u_item.head()

,movieID,movie_title,release_date,video_release,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action|Adventure|Thriller
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action|Comedy|Drama
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime|Drama|Thriller
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama


In [79]:
# получим рекомендованные фильмы
for movieID, _ in series_scores[:5].items():
    row = u_item.loc[u_item['movieID'] == movieID]
    s = row['movie_title']
    print(s)

243    Devil's Own, The (1997)
Name: movie_title, dtype: object
326    Conspiracy Theory (1997)
Name: movie_title, dtype: object
300    L.A. Confidential (1997)
Name: movie_title, dtype: object
330    Kiss the Girls (1997)
Name: movie_title, dtype: object
319    Mother (1996)
Name: movie_title, dtype: object


In [67]:
# векторное представление фильма
_, np_item_embeddings = model.get_item_representations(features=sm_item_features)
print(np_item_embeddings.shape)
np_item_embeddings[:2]

(1681, 10)


array([[ 0.10753414, -0.20126322,  0.03345471, -0.3057706 ,  0.1114855 ,
        -0.38210753, -0.21377389, -0.3980296 ,  0.49922514,  0.25208032],
       [ 0.08947685,  0.18916625,  0.11129572,  0.01311499, -0.2054962 ,
         0.11450414, -0.26834697,  0.0335913 , -0.34658822, -0.20515837]],
      dtype=float32)

In [68]:
# вычисляем косинусное сходство для каждой пары фильмов
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import sparse

np_item_similarities = cosine_similarity(sparse.csr_matrix(np_item_embeddings))
print(np_item_similarities.shape)
np_item_similarities[:2]

(1681, 1681)


array([[ 0.99999994, -0.5170222 ,  0.05688985, ..., -0.33512828,
        -0.26820168, -0.5395706 ],
       [-0.5170222 ,  1.        , -0.19536582, ...,  0.3218555 ,
         0.48893926,  0.10063438]], dtype=float32)

In [69]:
# сортируем оценки в порядке убывания
series_scores.sort_values(ascending=False, inplace=True)
series_scores[:5]

,0
245,7.220359
328,4.510055
302,4.432845
332,4.330058
321,4.066901


In [70]:
_, np_item_embeddings = model.get_item_representations(features=sm_item_features)
print(np_item_embeddings.shape)
np_item_embeddings[:2]

(1681, 10)


array([[ 0.10753414, -0.20126322,  0.03345471, -0.3057706 ,  0.1114855 ,
        -0.38210753, -0.21377389, -0.3980296 ,  0.49922514,  0.25208032],
       [ 0.08947685,  0.18916625,  0.11129572,  0.01311499, -0.2054962 ,
         0.11450414, -0.26834697,  0.0335913 , -0.34658822, -0.20515837]],
      dtype=float32)

##Установка TensorFlow

In [2]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorboard
    Found existing installation

In [3]:
!pip install tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np

##Анализ датасета

In [49]:
# загрузим MovieLens
ratings, ratings_info = tfds.load('movielens/100k-ratings',
                                     with_info=True,
                                     split='train')
movies, movies_info = tfds.load('movielens/100k-movies',
                                    with_info=True,
                                    split='train')

In [50]:
# загрузим данные
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

In [51]:
# посмотрим информацию о датасете
movies_info

tfds.core.DatasetInfo(
    name='movielens',
    full_name='movielens/100k-movies/0.1.1',
    description="""
    This dataset contains a set of movie ratings from the MovieLens website, a movie
    recommendation service. This dataset was collected and maintained by [GroupLens]
    (https://grouplens.org/), a research group at the University of Minnesota. There
    are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
    datasets, the movies data and ratings data are joined on "movieId". The 25m
    dataset, latest-small dataset, and 20m dataset contain only movie data and
    rating data. The 1m dataset and 100k dataset contain demographic data in
    addition to movie and rating data.
    
    - "25m": This is the latest stable version of the MovieLens dataset. It is
    recommended for research purposes.
    - "latest-small": This is a small subset of the latest version of the MovieLens
    dataset. It is changed and updated over time by GroupLens.
    - "100

In [52]:
#посмотреть можно так
ds_r = tfds.as_dataframe(ratings.take(5), ratings_info)
ds_r

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,7 (Drama),b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4 (doctor/health care),b'doctor',4.0,b'53211'
1,25.0,4 (Comedy)14 (Romance),b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5 (entertainment),b'entertainment',2.0,b'80525'
2,18.0,4 (Comedy),b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17 (student),b'student',4.0,b'55439'
3,50.0,5 (Crime)7 (Drama),b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4 (doctor/health care),b'healthcare',4.0,b'06472'
4,50.0,10 (Horror)16 (Thriller),b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18 (technician/engineer),b'technician',3.0,b'75094'


In [53]:
ds_m = tfds.as_dataframe(movies.take(5), movies_info)
ds_m

,movie_genres,movie_id,movie_title
0,4 (Comedy),b'1681',b'You So Crazy (1994)'
1,4 (Comedy)7 (Drama),b'1457',b'Love Is All There Is (1996)'
2,1 (Adventure)3 (Children),b'500',b'Fly Away Home (1996)'
3,0 (Action),b'838',b'In the Line of Duty 2 (1987)'
4,7 (Drama),b'1648',"b'Niagara, Niagara (1997)'"


In [54]:
# посмотрим набор данных рейтингов
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [55]:
# посмотрим набор данных о фильмах
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


##TensorFlow Recommenders user_id и movie_title. Вариант 1
https://www.tensorflow.org/recommenders/examples/basic_retrieval?hl=ru

In [56]:
# в 1-й моделе сохраним только user_id и movie_title в наборе данных
ratings = ratings.map(lambda x: {'movie_title': x['movie_title'],
                                 'user_id': x['user_id']
})
movies = movies.map(lambda x: x['movie_title'])

In [57]:
# посмотрим
tfds.as_dataframe(ratings.take(5))

,movie_title,user_id
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,b'Strictly Ballroom (1992)',b'92'
2,"b'Very Brady Sequel, A (1996)'",b'301'
3,b'Pulp Fiction (1994)',b'60'
4,b'Scream 2 (1997)',b'197'


In [58]:
# посмотрим
tfds.as_dataframe(movies.take(5))

,
0,b'You So Crazy (1994)'
1,b'Love Is All There Is (1996)'
2,b'Fly Away Home (1996)'
3,b'In the Line of Duty 2 (1987)'
4,"b'Niagara, Niagara (1997)'"


In [59]:
# разобъем данные train и test
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [60]:
# посморим
tfds.as_dataframe(train.take(5)), tfds.as_dataframe(test.take(5))

(                     movie_title user_id
 0         b'Postman, The (1997)'  b'681'
 1             b'Clueless (1995)'  b'442'
 2  b'Maltese Falcon, The (1941)'  b'932'
 3      b'His Girl Friday (1940)'  b'506'
 4            b'Quiz Show (1994)'   b'18',
                        movie_title user_id
 0                b'M*A*S*H (1970)'  b'346'
 1                b'Volcano (1997)'  b'602'
 2  b'2001: A Space Odyssey (1968)'  b'393'
 3     b'Dances with Wolves (1990)'  b'152'
 4                  b'Speed (1994)'  b'738')

In [61]:
# найдем уникальные идентификаторы пользователей и названия фильмов, присутствующие в данных
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# посмотрим
unique_movie_titles[:10], unique_user_ids[:10]

(array([b"'Til There Was You (1997)", b'1-900 (1994)',
        b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
        b'2 Days in the Valley (1996)',
        b'20,000 Leagues Under the Sea (1954)',
        b'2001: A Space Odyssey (1968)',
        b'3 Ninjas: High Noon At Mega Mountain (1998)',
        b'39 Steps, The (1935)'], dtype=object),
 array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
        b'106', b'107'], dtype=object))

Построим модель

In [62]:
# зададим размерность
embedding_dimension = 32

In [63]:
# построим модель пользователей
user_model = tf.keras.Sequential([
             tf.keras.layers.StringLookup(
             vocabulary=unique_user_ids, mask_token=None),
  # добавляем дополнительное встраивание для учета неизвестных токенов
             tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [64]:
# построим модель фильмов
movie_model = tf.keras.Sequential([
              tf.keras.layers.StringLookup(
              vocabulary=unique_movie_titles, mask_token=None),
              tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [65]:
# зададим метрику
metrics = tfrs.metrics.FactorizedTopK(      # обязательно tensorflow==2.15.0
          candidates=movies.batch(128).map(movie_model)
)

In [66]:
# зададим функцию потерь
task = tfrs.tasks.Retrieval(metrics=metrics)

In [67]:
# соберем модель целиком
#сделать это рекомендуется так

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # выбираем пользовательские функции и передаем их в пользовательскую модель
    user_embeddings = self.user_model(features['user_id'])
    # затем выберите элементы фильма и передайте их в модель фильма, возвращая вложения обратно
    positive_movie_embeddings = self.movie_model(features['movie_title'])

    # возвращаем потери и показатели
    return self.task(user_embeddings, positive_movie_embeddings)

In [68]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # установим градиентную ленту для записи градиентов
    with tf.GradientTape() as tape:

      # расчет потерь
      user_embeddings = self.user_model(features['user_id'])
      positive_movie_embeddings = self.movie_model(features['movie_title'])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics['loss'] = loss
    metrics['regularization_loss'] = regularization_loss
    metrics['total_loss'] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # расчет потерь
    user_embeddings = self.user_model(features['user_id'])
    positive_movie_embeddings = self.movie_model(features['movie_title'])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # потерями при регуляризации
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics['loss'] = loss
    metrics['regularization_loss'] = regularization_loss
    metrics['total_loss'] = total_loss

    return metrics

In [93]:
# создадим экземпляр модели
model = MovielensModel(user_model, movie_model)
learning_rate=0.05
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))
type(model)

__main__.MovielensModel

In [94]:
# подготовим данные
cached_train = train.shuffle(100_000).batch(8_192).cache()
cached_test = test.batch(4_096).cache()

In [76]:
# обучим модель
epochs = 16
model.fit(cached_train, epochs=epochs)

Epoch 1/16
10/10 [==============================] - 34s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0050 - factorized_top_k/top_5_categorical_accuracy: 0.0309 - factorized_top_k/top_10_categorical_accuracy: 0.0600 - factorized_top_k/top_50_categorical_accuracy: 0.2228 - factorized_top_k/top_100_categorical_accuracy: 0.3552 - loss: 65254.6676 - regularization_loss: 0.0000e+00 - total_loss: 65254.6676
Epoch 2/16
10/10 [==============================] - 31s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0049 - factorized_top_k/top_5_categorical_accuracy: 0.0319 - factorized_top_k/top_10_categorical_accuracy: 0.0606 - factorized_top_k/top_50_categorical_accuracy: 0.2256 - factorized_top_k/top_100_categorical_accuracy: 0.3589 - loss: 65084.1200 - regularization_loss: 0.0000e+00 - total_loss: 65084.1200
Epoch 3/16
10/10 [==============================] - 32s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0050 - factorized_top_k/top_5_categorical_accuracy: 0.0

In [77]:
# получим предсказания
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 11s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0111 - factorized_top_k/top_50_categorical_accuracy: 0.0896 - factorized_top_k/top_100_categorical_accuracy: 0.1916 - loss: 31565.0306 - regularization_loss: 0.0000e+00 - total_loss: 31565.0306


{'factorized_top_k/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_5_categorical_accuracy': 0.003650000086054206,
 'factorized_top_k/top_10_categorical_accuracy': 0.011099999770522118,
 'factorized_top_k/top_50_categorical_accuracy': 0.089649997651577,
 'factorized_top_k/top_100_categorical_accuracy': 0.19155000150203705,
 'loss': 28655.77734375,
 'regularization_loss': 0,
 'total_loss': 28655.77734375}

In [78]:
# создадим модель
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# рекомендуем фильмы из всего набора
index.index_from_dataset(tf.data.Dataset.zip((movies.batch(100),
                                              movies.batch(100).map(model.movie_model)))
)

# получим рекомендации
_, titles = index(tf.constant(['42']))
print(f'Recommendations for user 42: {titles[0, :5]}')

Recommendations for user 42: [b'Rent-a-Kid (1995)' b'Mirror Has Two Faces, The (1996)' b'Rudy (1993)'
 b'Pinocchio (1940)' b'Michael (1996)']


In [79]:
_

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[3.1330059, 2.9720678, 2.961512 , 2.8727176, 2.732448 , 2.7109613,
        2.704308 , 2.682961 , 2.6739504, 2.6509192]], dtype=float32)>

##TensorFlow Recommenders. Фильмы/Рейтинги. Вариант 2
https://www.tensorflow.org/recommenders/examples/basic_ranking?hl=ru

In [64]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
import tensorflow_recommenders as tfrs

In [5]:
# загрузим данные
ratings = tfds.load('movielens/100k-ratings', split='train')

ratings = ratings.map(lambda x: {'movie_title': x['movie_title'],
                                 'user_id': x['user_id'],
                                 'user_rating': x['user_rating']
})

In [6]:
# загрузим данные
ratings = tfds.load('movielens/100k-ratings', split='train')

ratings = ratings.map(lambda x: {'movie_title': x['movie_title'],
                                 'user_id': x['user_id'],
                                 'user_rating': x['user_rating']
})

In [7]:
# разделим на test и train
# случайным образом
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [9]:
# найдем уникальные идентификаторы пользователей и названия фильмов, присутствующие в данных
movie_titles = ratings.batch(1000000).map(lambda x: x['movie_title'])
user_ids = ratings.batch(1000000).map(lambda x: x['user_id'])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [10]:
# посмотрим
unique_movie_titles[:10], unique_user_ids[:10]

(array([b"'Til There Was You (1997)", b'1-900 (1994)',
        b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
        b'2 Days in the Valley (1996)',
        b'20,000 Leagues Under the Sea (1954)',
        b'2001: A Space Odyssey (1968)',
        b'3 Ninjas: High Noon At Mega Mountain (1998)',
        b'39 Steps, The (1935)'], dtype=object),
 array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
        b'106', b'107'], dtype=object))

In [23]:
# построим модель
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # зададим embeddings для пользователей
    self.user_embeddings = tf.keras.Sequential([tf.keras.layers.StringLookup(
                                                vocabulary=unique_user_ids,
                                                mask_token=None),
                           tf.keras.layers.Embedding(len(unique_user_ids) + 1,
                                                     embedding_dimension)
    ])

    # зададим embeddings для фильмов
    self.movie_embeddings = tf.keras.Sequential([tf.keras.layers.StringLookup(
                                                 vocabulary=unique_movie_titles,
                                                 mask_token=None),
                            tf.keras.layers.Embedding(len(unique_movie_titles) + 1,
                                                      embedding_dimension)
    ])

    # зададим плотные слои
    self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation='relu'),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [13]:
# зададим функцию потерь и метрику
task = tfrs.tasks.Ranking(loss = tf.keras.losses.MeanSquaredError(),
                          metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [24]:
# соберем модель
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features['user_id'], features['movie_title']))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop('user_rating')

    rating_predictions = self(features)

    return self.task(labels=labels, predictions=rating_predictions)

In [25]:
# создадим экземпляр класса модели
model = MovielensModel()
learning_rate = 0.05
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate))

In [26]:
# перемешаем и кэшируем даные для оценки и обучения
cached_train = train.shuffle(100_000).batch(8_192).cache()
cached_test = test.batch(4_096).cache()

In [27]:
# обучим модель
model.fit(cached_train, epochs=16)

Epoch 1/16
10/10 [==============================] - 5s 51ms/step - root_mean_squared_error: 1.9264 - loss: 3.4374 - regularization_loss: 0.0000e+00 - total_loss: 3.4374
Epoch 2/16
10/10 [==============================] - 0s 40ms/step - root_mean_squared_error: 1.1348 - loss: 1.2808 - regularization_loss: 0.0000e+00 - total_loss: 1.2808
Epoch 3/16
10/10 [==============================] - 0s 37ms/step - root_mean_squared_error: 1.1190 - loss: 1.2465 - regularization_loss: 0.0000e+00 - total_loss: 1.2465
Epoch 4/16
10/10 [==============================] - 0s 40ms/step - root_mean_squared_error: 1.1113 - loss: 1.2294 - regularization_loss: 0.0000e+00 - total_loss: 1.2294
Epoch 5/16
10/10 [==============================] - 0s 36ms/step - root_mean_squared_error: 1.1038 - loss: 1.2125 - regularization_loss: 0.0000e+00 - total_loss: 1.2125
Epoch 6/16
10/10 [==============================] - 0s 40ms/step - root_mean_squared_error: 1.0942 - loss: 1.1911 - regularization_loss: 0.0000e+00 - total

In [28]:
# оценим модель на тесте
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 17ms/step - root_mean_squared_error: 0.9870 - loss: 0.9728 - regularization_loss: 0.0000e+00 - total_loss: 0.9728


{'root_mean_squared_error': 0.9870291948318481,
 'loss': 0.9665654897689819,
 'regularization_loss': 0,
 'total_loss': 0.9665654897689819}

In [32]:
# получаем рейтинги для фильма для данного пользователя
test_ratings = {}
test_movie_titles = ['M*A*S*H (1970)', 'Dances with Wolves (1990)', 'Speed (1994)']
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      'user_id': np.array(['48']),
      'movie_title': np.array([movie_title])
  })

print('Ratings:')
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[4.0609865]]
Dances with Wolves (1990): [[3.9092896]]
Speed (1994): [[3.740467]]
